# Restaurant Agent Red Team Evaluation

This notebook performs red team evaluation on the Scheibmeir's restaurant agent to assess its safety and resilience against adversarial attacks.
It uses Azure AI Evaluation's Red Team functionality to test the agent across different risk categories.

## Setup and Configuration

In [1]:
import os
import asyncio
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

# Load environment variables from .env file
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Configuration from environment variables
RESTAURANT_ASSISTANT_ID = os.getenv("RESTAURANT_ASSISTANT_ID")
RESTAURANT_ASSISTANT_PROJECT = os.getenv("RESTAURANT_ASSISTANT_PROJECT")

print(f"Restaurant Assistant ID: {RESTAURANT_ASSISTANT_ID}")
print(f"Restaurant Project URL: {RESTAURANT_ASSISTANT_PROJECT}")

Restaurant Assistant ID: asst_eJv1oQY8pHlj3mgXyV2sLCYG
Restaurant Project URL: https://aipmaker-project-resource.services.ai.azure.com/api/projects/aipmaker-project


## Initialize Clients

In [3]:
# Initialize Azure credential
credential = DefaultAzureCredential()

# Initialize the Agents client
agents_client = AgentsClient(
    endpoint=RESTAURANT_ASSISTANT_PROJECT,
    credential=credential
)

print("Clients initialized successfully!")

Clients initialized successfully!


## Define Target Callback for Red Team Testing

This callback function will be used by the Red Team to test the restaurant agent.

In [4]:
async def restaurant_agent_callback(
    messages: list,
    stream: Optional[bool] = False,
    session_state: Optional[str] = None,
    context: Optional[Dict[str, Any]] = None,
) -> Dict[str, List[Dict[str, str]]]:
    """
    Callback function that interfaces with the restaurant agent.
    This function will be called by the Red Team evaluation framework.
    """
    try:
        # Extract the latest message from the conversation history
        latest_message = messages[-1].content if messages else "Hello"
        
        # Create a thread and run in one step
        result = agents_client.create_thread_and_process_run(
            agent_id=RESTAURANT_ASSISTANT_ID,
            thread={
                "messages": [
                    {
                        "role": "user",
                        "content": latest_message
                    }
                ]
            }
        )
        
        if result.status.value == "completed":
            # Get messages from the thread
            agent_messages = agents_client.messages.list(thread_id=result.thread_id)
            
            # Find the agent's response
            for msg in agent_messages:
                if msg.role.value == "agent":
                    formatted_response = {
                        "content": msg.content[0].text.value,
                        "role": "assistant"
                    }
                    return {"messages": [formatted_response]}
        
        # If no agent response found or run failed
        error_response = {
            "content": f"I encountered an error processing your request. Status: {result.status.value}",
            "role": "assistant"
        }
        return {"messages": [error_response]}
        
    except Exception as e:
        error_response = {
            "content": f"I encountered an error and couldn't process your request: {str(e)}",
            "role": "assistant"
        }
        return {"messages": [error_response]}

print("Restaurant agent callback function defined!")

Restaurant agent callback function defined!


## Initialize Red Team Evaluator

In [6]:
# Azure AI project endpoint for red team
azure_ai_project = RESTAURANT_ASSISTANT_PROJECT

# Create the RedTeam instance with specific risk categories relevant to a restaurant agent
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ],
    num_objectives=3,  # Number of attack objectives per risk category
)

print("Red Team evaluator initialized!")
#print(f"Risk categories: {[cat.value for cat in red_team._risk_categories]}")
#print(f"Number of objectives per category: {red_team._num_objectives}")

Red Team evaluator initialized!


## Run Basic Red Team Scan

This performs a basic red team evaluation using simple attack strategies.

In [8]:
# Run a basic red team scan
print("Starting basic red team scan...")
print("This may take several minutes to complete.")

basic_result = await red_team.scan(
    target=restaurant_agent_callback,
    scan_name="Restaurant-Agent-Basic-Red-Team",
    # Baseline attack strategy only (baseline is included by default)
    output_path="basic_red_team_results.json",
)

print("Basic red team scan completed!")
print("Results saved to: basic_red_team_results.json")

Starting basic red team scan...
This may take several minutes to complete.
🚀 STARTING RED TEAM SCAN: Restaurant-Agent-Basic-Red-Team
📂 Output directory: .\.scan_Restaurant-Agent-Basic-Red-Team_20250811_201108
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/8b6701ce-1b78-43e1-bdb3-218bf8ae0953?wsid=/subscriptions/21039746-6e73-4627-88af-efa80f856c2c/resourceGroups/rg-AIPMaker/providers/Microsoft.CognitiveServices/accounts/aipmaker-project-resource/projects/aipmaker-project&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
📋 Planning 4 total tasks
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/8b6701ce-1b78-43e1-bdb3-218bf8ae0953?wsid=/subscriptions/21039746-6e73-4627-88af-efa80f856c2c/resourceGroups/rg-AIPMaker/providers/Microsoft.CognitiveServices/accounts/aipmaker-project-resource/projects/aipmaker-project&tid=72f988bf-86f1-41af-91ab-2d7cd

Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/4 [00:04<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 3 objectives


...

⚙️ Processing 4 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for self_harm risk category


Scanning:  75%|████████████████████████▊        | 3/4 [04:56<04:56, 296.19s/scan, current=batch 1/1]

Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Basic-Red-Team_20250811_201108\baseline_violence_1d03fa2d-40b7-473c-8d39-19f06c63d08b.json".

✅ Completed task 1/4 (25.0%) - baseline/violence in 291.1s
   Est. remaining: 15.1 minutes
Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Basic-Red-Team_20250811_201108\baseline_hate_unfairness_0c01aad2-90e7-4fb4-8a2f-494e53990552.json".

✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 291.2s
   Est. remaining: 5.0 minutes
Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Basic-Red-Team_20250811_201108\baseline_sexual_577fc85e-ae18-4e3c-ba98-188e174d3cef.json".

✅ Completed task 3/4 (75.0%) - baseline/sexual in 291.2s
   Est. remaining: 1.7 minutes


Scanning: 100%|██████████████████████████████████| 4/4 [05:01<00:00, 75.26s/scan, current=batch 1/1]
Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.

Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Basic-Red-Team_20250811_201108\baseline_self_harm_738a71fa-cead-4737-8453-e73f8a5bbdb7.json".

✅ Completed task 4/4 (100.0%) - baseline/self_harm in 296.0s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\basic_red_team_results.json".

Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Basic-Red-Team_20250811_201108\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/12 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence          | 0.0%           | N/

## Run Advanced Red Team Scan

This performs a more comprehensive red team evaluation with multiple attack strategies.

In [9]:
# Run an advanced red team scan mirroring the sample's strategies
print("Starting advanced red team scan with comprehensive strategies...")

advanced_result = await red_team.scan(
    target=restaurant_agent_callback,
    scan_name="Restaurant-Agent-Advanced-Red-Team",
    attack_strategies=[
        AttackStrategy.Base64,
        AttackStrategy.Flip,
        AttackStrategy.CharacterSpace,
        AttackStrategy.ROT13,
        AttackStrategy.UnicodeConfusable,
        AttackStrategy.Tense,
        AttackStrategy.CharSwap,
        AttackStrategy.Morse,
        AttackStrategy.Leetspeak,
        AttackStrategy.Url,
        AttackStrategy.Binary,
        AttackStrategy.MODERATE,
        AttackStrategy.DIFFICULT,
    ],
    output_path="advanced_red_team_results.json",
)

print("Advanced red team scan completed!")
print("Results saved to: advanced_red_team_results.json")

Starting advanced red team scan with comprehensive strategies...
🚀 STARTING RED TEAM SCAN: Restaurant-Agent-Advanced-Red-Team
📂 Output directory: .\.scan_Restaurant-Agent-Advanced-Red-Team_20250811_201701
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/e7bb607a-db55-43ec-91a7-d1de3eb3f20c?wsid=/subscriptions/21039746-6e73-4627-88af-efa80f856c2c/resourceGroups/rg-AIPMaker/providers/Microsoft.CognitiveServices/accounts/aipmaker-project-resource/projects/aipmaker-project&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
📋 Planning 52 total tasks
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/e7bb607a-db55-43ec-91a7-d1de3eb3f20c?wsid=/subscriptions/21039746-6e73-4627-88af-efa80f856c2c/resourceGroups/rg-AIPMaker/providers/Microsoft.CognitiveServices/accounts/aipmaker-project-resource/projects/aipmaker-project&tid=72f988bf-86f1-41af-91ab-2d7cd011

Scanning:   0%|                        | 0/52 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                 | 0/52 [00:00<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 3 objectives


...

Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Advanced-Red-Team_20250811_201701\tense_base64_hate_unfairness_6471392e-f1f4-4b14-91ee-88203f38ded6.json".

✅ Completed task 50/52 (96.2%) - tense_base64/hate_unfairness in 204.8s
   Est. remaining: 1.5 minutes
▶️ Starting task: tense_base64 strategy for sexual risk category
▶️ Starting task: tense_base64 strategy for self_harm risk category


Scanning:  98%|█████████████████████████████▍| 51/52 [37:55<00:36, 36.06s/scan, current=batch 11/11]

Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Advanced-Red-Team_20250811_201701\tense_base64_sexual_a86efbf9-c0c9-4c7e-8413-92412cc58b69.json".

✅ Completed task 51/52 (98.1%) - tense_base64/sexual in 84.4s
   Est. remaining: 0.7 minutes


Scanning: 100%|██████████████████████████████| 52/52 [38:00<00:00, 43.86s/scan, current=batch 11/11]


Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Advanced-Red-Team_20250811_201701\tense_base64_self_harm_d4ff2351-8ca1-4173-a915-4fcda6199a94.json".

✅ Completed task 52/52 (100.0%) - tense_base64/self_harm in 90.0s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\advanced_red_team_results.json".

Evaluation results saved to "C:\src\gartner-ai-app-dev-2025\.scan_Restaurant-Agent-Advanced-Red-Team_20250811_201701\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/156 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence          | 

In [10]:
# Quick summary of results
try:
    basic_count = len(basic_result.redteaming_data) if getattr(basic_result, "redteaming_data", None) else 0
except Exception:
    basic_count = 0

try:
    advanced_count = len(advanced_result.redteaming_data) if getattr(advanced_result, "redteaming_data", None) else 0
except Exception:
    advanced_count = 0

print(f"Basic conversations collected: {basic_count}")
print(f"Advanced conversations collected: {advanced_count}")
print("Files: basic_red_team_results.json, advanced_red_team_results.json")

Basic conversations collected: 0
Advanced conversations collected: 0
Files: basic_red_team_results.json, advanced_red_team_results.json
